In [4]:
%load_ext autoreload
%autoreload 2

import torch

torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from text3d2video.util import read_obj_with_uvs
from text3d2video.utilities.camera_placement import (
    cam_view_prompt,
    turntable_extrinsics,
)
from pytorch3d.renderer import FoVPerspectiveCameras

mesh_path = "data/meshes/mixamo-human.obj"
device = "cuda"

# read mesh
mesh, verts_uvs, faces_uvs = read_obj_with_uvs(mesh_path, device=device)

n_frames = 10
dist = 6

angles = torch.linspace(0, 360, n_frames)
R, T = turntable_extrinsics(dists=dist, angles=angles)
frame_cams = FoVPerspectiveCameras(R=R, T=T, device="cuda", fov=10)

prompt_suffixes = [f", {cam_view_prompt(a, 0)} view" for a in angles]
frame_meshes = mesh.extend(len(frame_cams))

In [ ]:
from text3d2video.rendering import render_depth_map


render_depth_map(frame_cams, frame_meshes, device=device, n_frames=n_frames)